In [8]:
!pip install spektral

You should consider upgrading via the 'C:\Users\ricar\Desktop\nemProjectPython\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [1]:
from spektral.datasets.citation import Citation
from spektral.transforms import LayerPreprocess
from spektral.layers import GCNConv

In [2]:
dataset = Citation("cora", normalize_x=True, transforms=
                   [LayerPreprocess(GCNConv)]);

Pre-processing node features


C:\Users\ricar\Desktop\nemProjectPython\venv\lib\site-packages\spektral\datasets\citation.py:108: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  a = nx.adjacency_matrix(nx.from_dict_of_lists(graph))  # CSR
C:\Users\ricar\Desktop\nemProjectPython\venv\lib\site-packages\scipy\sparse\_index.py:146: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [3]:
dataset.graphs[0]

Graph(n_nodes=2708, n_node_features=1433, n_edge_features=None, n_labels=7)

In [4]:
import numpy as np
def mascarar(mask):
  return mask.astype(np.float32)/np.count_nonzero(mask)

pesos_treino, pesos_val, pesos_test = (
    mascarar(mask) for
    mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
)
print("Tamanho do treino: ",
      len([x for x in dataset.mask_tr if x]))
print("Tamanho da validação: ",
      len([x for x in dataset.mask_va if x]))
print("Tamanho do teste: ", len([x for x in dataset.mask_te if x]))

Tamanho do treino:  140
Tamanho da validação:  500
Tamanho do teste:  1000


In [9]:
from spektral.models.gcn import GCN
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from spektral.data.loaders import SingleLoader

model = GCN(n_labels=dataset.n_labels)

carregador_treino = SingleLoader(dataset, sample_weights=pesos_treino)
carregador_val = SingleLoader(dataset, sample_weights=pesos_val)

In [ ]:
epocas = 20
espera = 10
from tensorflow.keras.callbacks import EarlyStopping

model.compile(optimizer=Adam(1e-2),
              loss=CategoricalCrossentropy(reduction="sum"),
              weighted_metrics=["acc"])
model.fit(
    carregador_treino.load(),
    steps_per_epoch=carregador_treino.steps_per_epoch,
    validation_data = carregador_val.load(),
    validation_steps=carregador_val.steps_per_epoch,
    epochs=epocas,
    callbacks=[EarlyStopping(patience=espera, restore_best_weights=True)]
)

In [ ]:
carregador_teste = SingleLoader(dataset, sample_weights=pesos_test)

result = model.evaluate(carregador_teste.load(),
                        steps=carregador_teste.steps_per_epoch)

print(result)